<a href="https://colab.research.google.com/github/mengjie514/MultiLabel-Text-Classification/blob/main/NN_MultiLabels_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Pre-processing**

In [2]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/My Drive/Colab Notebooks/ETC")
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', -1)

df = pd.read_csv('haha_raw.csv') #Qualification.sav
df = df[['string.Education', 'QualType']]

df['QualType'] = df['QualType'].astype(str)
df['string.Education'] = df['string.Education'].astype(str)
df.reset_index(inplace=True)
df_words = df

from sklearn.preprocessing import MultiLabelBinarizer
annotation_new=[]

for cell in df['QualType']:
    cell=cell.replace(" ", "") 
    cell=cell.replace("&", "& ") 
    annotation_new.append(cell.split(","))

df['annotation_new'] = annotation_new 

mlb = MultiLabelBinarizer() 
binary_labels=pd.DataFrame(mlb.fit_transform(df['annotation_new']),columns=mlb.classes_) 
binary_labels=binary_labels.sort_index(axis=1) 

df = df.join(binary_labels)

# Flexible recodes
df['level1-4'] = df[['2.0', '3.0', '4.0', '5.0']].max(axis=1)
df['level5-6'] = df[['6.0', '7.0']].max(axis=1)
df['invalidresponse']=df[['13.0', '14.0', '16.0', '17.0', '18.0', 'nan']].max(axis=1)

df = df[['string.Education', 'level1-4', 'level5-6', '8.0', '9.0', '10.0', '11.0', '12.0', '1.0', 'invalidresponse']]
df = df.rename(columns = {'8.0':'Level7BachelorDegree', "9.0": "PostgraduateBacheloHonours",
                          "10.0": "MastersDegree", "11.0": "DoctorateDegree",
                          "12.0": "OverseasSecondarySchoolQualification", "1.0":"NoQualification"})

labels = ['level1-4', 'level5-6', 'Level7BachelorDegree', 'PostgraduateBacheloHonours', 'MastersDegree',  'DoctorateDegree', 'OverseasSecondarySchoolQualification', 'NoQualification', 'invalidresponse']
df['string.Education'] = df['string.Education'].astype(str)

df.head(5)

Mounted at /content/gdrive


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,string.Education,level1-4,level5-6,Level7BachelorDegree,PostgraduateBacheloHonours,MastersDegree,DoctorateDegree,OverseasSecondarySchoolQualification,NoQualification,invalidresponse
0,,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,1,0
2,0 4TH FORM,0,0,0,0,0,0,0,1,0
3,0 5,0,0,0,0,0,0,0,0,1
4,0 GRADE,0,0,0,0,0,0,1,0,0


**2. Text Cleaning**

In [3]:
import string
from html.parser import HTMLParser
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer, word_tokenize
#from autocorrect import spell 

# Flexible recodes
split_adhere_dic = {
    "BA": "BACHELOR OF ART", "BACH": "BACHELOR",
    "B A": "BACHELOR OF ART", "B ART": "BACHELOR OF ART", "B ARTS": "BACHELOR OF ARTS",
    "B SC": "BACHELOR OF SCIENCE", "B S": "BACHELOR OF SCIENCE",
    "B COM": "BACHELOR OF COM", "B  COMM": "BACHELOR OF COMM", "B OF COMM": "BACHELOR OF COMM", 
    "B  COMMERCE": "BACHELOR OF COMMERCE", 
    "B AG": "BACHELOR OF AG", "B AGR": "BACHELOR OF AGR", "B AGRI": "BACHELOR OF AGRI", 
    "B AP": "BACHELOR OF AP", "B APP": "BACHELOR OF APP", "B APPL": "BACHELOR OF APPL", 
    "B APPLIED": "BACHELOR OF APPLIED",
    "B ARCH": "BACHELOR OF ARCH", 
    "B B": "BACHELOR OF B", "B D": "BACHELOR OF D", "B H": "BACHELOR OF H", "B HORT": "BACHELOR OF HORT",
    "B MUS": "BACHELOR OF MUS", "B HSC": "BACHELOR OF HSC", 
    "B TCH": "BACHELOR OF TCH", "B TCHING": "BACHELOR OF TCHING", "B TEACHING": "BACHELOR OF TEACHING",
    "B TECH": "BACHELOR OF TECH",
    "B BUS": "BACHELOR OF BUS", "B BUSINESS": "BACHELOR OF BUSINESS",
    "B E": "BACHELOR OF E", "B ENG": "BACHELOR OF ENG", "B ED": "BACHELOR OF ED", "B  EDUC": "BACHELOR OF EDUC", 
    "B SOC": "BACHELOR OF SOC", "L L B": "LLB", "B OF": "BACHELOR OF", 
    
    "MA": "MASTER OF ARTS", "M A": "MASTER OF ARTS", "M ARTS": "MASTER OF ARTS", 
    "M S": "MASTER OF SCIENCE", "M SC": "MASTER OF SCIENCE", "MSC": "MASTER OF SCIENCE",
    "M AG": "MASTER OF AG", "M AGR": "MASTER OF AGR", "M AGRI": "MASTER OF AGRI", 
    "M AP": "MASTER OF AP", "M APP": "MASTER OF APP", "M APPL": "MASTER OF APPL", "M APPLIED": "MASTER OF APPLIED",
    "M B A": "MBA", "M BA": "MBA",
    "M C": "MASTER OF CM", "M COM": "MASTER OF COM", "M  COMM": "MASTER OF COMM", "M OF COMM": "MASTER OF COMM", 
    "M  COMMERCE": "MASTER OF COMMERCE", 
    "M D": "MASTER OF D", "M ENG": "MASTER OF ENG", "M  EDUC": "MASTER OF EDUC",
    "M E": "MASTER OF E", "ME": "MASTER OF E", "MENG": "MASTER OF E", 
    "M ED": "MASTER OF ED", "MED": "MASTER OF ED", "M MED": "MASTER OF MED",
    "M TCHING": "MASTER OF TCHING", "M PHIL": "MPHIL",  "M SOC": "MASTER OF SOC", 
    
    "P H D": "PHD", "PH D": "PHD", "PHDC": "PHD C", "PH": "PHD", "D PHIL": "DPHIL", "D TCHN": "PHD", 
    "POST DOC": "POSTDOC",
    "HONS": "HONOURS", "HON": "HONOURS", "HON S": "HONOURS", "HONORS": "HONOURS", 
    "NAT CERT": "NATIONAL CERTIFICATE", "DEG": "DEGREE", "DEGR": "DEGREE",
    
    "LVL": "LEVEL", "LV": 'LEVEL', "LVEL": "LEVEL",
    "L9": "LEVEL9", "L8": "LEVEL8", "L7": "LEVEL7", "L6": "LEVEL6", "L5": "LEVEL5", 
    "L4": "LEVEL4", "L3": "LEVEL3", "L2": "LEVEL2", "L1": "LEVEL1",
    "L 9": "LEVEL9", "L 8": "LEVEL8", "L 7": "LEVEL7", "L 6": "LEVEL6", "L 5": "LEVEL5", 
    "L 4": "LEVEL4", "L 3": "LEVEL3", "L 2": "LEVEL2", "L 1": "LEVEL1",
    "LV9": "LEVEL9", "LV8": "LEVEL8", "LV7": "LEVEL7", "LV6": "LEVEL6", "LV5": "LEVEL5", 
    "LV4": "LEVEL4", "LV3": "LEVEL3", "LV2": "LEVEL2", "LV1": "LEVEL1",
    "LVL9": "LEVEL9", "LVL8": "LEVEL8", "LVL7": "LEVEL7", "LVL6": "LEVEL6", "LVL5": "LEVEL5", 
    "LVL4": "LEVEL4", "LVL3": "LEVEL3", "LVL2": "LEVEL2", "LVL1": "LEVEL1",
    "LEVEL 9": "LEVEL9", "LEVEL 8": "LEVEL8", "LEVEL 7": "LEVEL7", "LEVEL 6": "LEVEL6", "LEVEL 5": "LEVEL5", 
    "LEVEL 4": "LEVEL4", "LEVEL 3": "LEVEL3", "LEVEL 2": "LEVEL2", "LEVEL 1": "LEVEL1",
    
    "ADVAN": "ADVANCED", "ADV": "ADVANCED",
    "YR": "YEAR", "YRS": "YEARS",
    "UNI": "UNIVERSITY",
    "U K": "UK"
    }

pat5 = r'\n'
pat6= r'\xa0'
combined_pat = r'|'.join((pat5, pat6))

split_pattern = re.compile(r'\b(' + '|'.join(split_adhere_dic.keys()) + r')\b')

def text_cleaner(demo):
    soup = BeautifulSoup(demo, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    splitted = re.sub(r"([a-z\.!?])([A-Z])", r"\1 \2", stripped) #split joined words
    split_handled = split_pattern.sub(lambda x: split_adhere_dic[x.group()], splitted)

    return split_handled

df['new_string_Education'] = [text_cleaner(t) for t in df['string.Education']]

# Remove stopwords/punctuations/numbers
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

table = str.maketrans('', '', string.punctuation)

def text_process(deep):
    words = re.split(r'\W+', deep)
    nopunc = [w.translate(table) for w in words]
    nostop =  ' '.join([word for word in nopunc if word.lower() not in stop_words])
    #nonum = ' '.join([word for word in str(nostop).split() if not word.isdigit()]) #remove numbers if needed
    return nostop

df['new_string_Education_deep'] = df.apply(lambda row: text_process(row.new_string_Education), axis=1)

# Stemming words
from nltk.stem import PorterStemmer, WordNetLemmatizer
porter_stemmer = PorterStemmer()

def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

df['new_string_Education_deep_stem'] = df['new_string_Education_deep'].apply(stem_sentences)
df = df[['new_string_Education_deep_stem', 'level1-4', 'level5-6', 'Level7BachelorDegree', 'PostgraduateBacheloHonours', 'MastersDegree',  'DoctorateDegree', 'OverseasSecondarySchoolQualification', 'NoQualification', 'invalidresponse']]

# Remove conflicts
df.loc[((df['new_string_Education_deep_stem'].str.contains('phd|master|honour|postgradu|postgrad|degre|degree|bachelor')) & 
        (df['level1-4'] == 1)), 'new_string_Education_deep_stem'] = df.loc[((df['new_string_Education_deep_stem'].str.contains('phd|master|honour|postgradu|postgrad|degre|degree|bachelor')) & 
                                                                            (df['level1-4'] == 1)), 'new_string_Education_deep_stem'].str.replace(r'phd|master|honour|postgradu|postgrad|degre|degree|bachelor', '')

df.loc[((df['new_string_Education_deep_stem'].str.contains('phd|master|honour|postgradu|postgrad|degre|degree|bachelor')) & 
        (df['level5-6'] == 1)), 'new_string_Education_deep_stem'] = df.loc[((df['new_string_Education_deep_stem'].str.contains('phd|master|honour|postgradu|postgrad|degre|degree|bachelor')) & 
                                                                            (df['level5-6'] == 1)), 'new_string_Education_deep_stem'].str.replace(r'phd|master|honour|postgradu|postgrad|degre|degree|bachelor', '')

df.loc[((df['new_string_Education_deep_stem'].str.contains('phd|master|honour|postgradu|postgrad|dipmola|certif')) & 
        (df['Level7BachelorDegree'] == 1)), 'new_string_Education_deep_stem'] = df.loc[((df['new_string_Education_deep_stem'].str.contains('phd|master|honour|postgradu|postgrad|dipmola|certif')) & 
                                                                                        (df['Level7BachelorDegree'] == 1)), 'new_string_Education_deep_stem'].str.replace(r'phd|master|honour|postgradu|postgrad|dipmola|certif', '')

df.loc[((df['new_string_Education_deep_stem'].str.contains('phd|degre|degree|bachelor|dipmola|certif')) & 
        (df['PostgraduateBacheloHonours'] == 1)), 'new_string_Education_deep_stem'] = df.loc[((df['new_string_Education_deep_stem'].str.contains('phd|degre|degree|bachelor|dipmola|certif')) & 
                                                                                              (df['PostgraduateBacheloHonours'] == 1)), 'new_string_Education_deep_stem'].str.replace(r'phd|degre|degree|bachelor|dipmola|certif', '')
df.head(5)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:111: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:115: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:119: FutureWarning: The default value of regex will change from True to False in a future version.
/

,new_string_Education_deep_stem,level1-4,level5-6,Level7BachelorDegree,PostgraduateBacheloHonours,MastersDegree,DoctorateDegree,OverseasSecondarySchoolQualification,NoQualification,invalidresponse
0,,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,1,0
2,0 4th form,0,0,0,0,0,0,0,1,0
3,0 5,0,0,0,0,0,0,0,0,1
4,0 grade,0,0,0,0,0,0,1,0,0


**3. Modelling**

In [4]:
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
import keras.models
from keras.models import Sequential, Model
from keras.layers import Embedding, SpatialDropout1D, Flatten, Dense, LSTM, GlobalMaxPool1D, Activation, Conv1D, Input, Bidirectional
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.layers.advanced_activations import Softmax
from tensorflow.keras.optimizers import Adam, SGD
from keras import backend as K
import tensorflow as tf
import gensim
from gensim.models import Word2Vec
from numpy import zeros
! pip install tensorflow-addons
import tensorflow_addons as tfa

maxlen = 50 # tested with 10 --> improve 50 --> no further improve
embedding_dims = 128
filters = 64
kernel_size = 3
max_features = 500

# Using Keras' Tokenizer to vectorizing a text corpus, with each text input being either converted 
# into integer sequence or a vector:
# (a) using fit_on_texts to create the vocabulary index based on word frequency,
# (b) texts_to_sequences to convert tokens of text to a sequence of integers, 
# with each word in the text being replaced by a corresponding integer value from the internal dictionary.
# or (c) converting tokens into a matrix form by texts_to_matrix.
tokenizer = Tokenizer(num_words=max_features, lower=True) # Extracting the most frequent words from corpus 
                                                          # (e.g., the top 500; tested with 50 --> improve 100 --> improve 500 --> no further improve)
tokenizer.fit_on_texts(df['new_string_Education_deep_stem']) # Vs. texts_to_matrix
sequences = tokenizer.texts_to_sequences(df['new_string_Education_deep_stem']) 
x = pad_sequences(sequences, maxlen=maxlen) # Ensure all sequences in a list have the same length 
                                            # (e.g., 20 for texts after removing all stopwords, special symbols; tested with 10 --> improve 20 --> no further improve)

seeds = [1, 43, 678, 90, 135]
X_train, X_test, y_train, y_test = train_test_split(x, 
                                                    df[df.columns[1:]],
                                                    test_size=0.3, 
                                                    random_state=seeds[0],
                                                    shuffle=True)

# Set class weights for imbalanced dataset and assign weights to training models
most_common_cat = pd.DataFrame()
most_common_cat['cat'] = df.columns[1:]
most_common_cat['count'] = df.iloc[:,1:].sum().values
most_common_cat.sort_values(['count'], inplace=True, ascending=False)
most_common_cat.reset_index(inplace=True, drop=True)

most_common_cat['class_weight'] = len(most_common_cat) / most_common_cat['count']
class_weight = {}
for index, label in enumerate(labels):
    class_weight[index] = most_common_cat[most_common_cat['cat'] == labels]['class_weight'].values[0]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 4.9 MB/s 


**3.1 Set up pipelines**

In [5]:
num_classes = y_train.shape[1]

def getModel_cnn():
    model = Sequential([
        Embedding(max_features, embedding_dims, input_length=maxlen),
        SpatialDropout1D(0.2),
        Conv1D(filters, kernel_size, padding='valid', activation='relu', strides=1), #relu for learning non-liner decision boundaries
        GlobalMaxPool1D(),
        Dense(num_classes, activation='sigmoid'),
    ],
    name="CNN_Model")
    return model

# LSTM as a type of recurrent neural network capable of learning sequences of observations
def getModel_lstm():
    model = Sequential([
        # First hiden layer to specify: 
        # output_dim (the size of output vectors from this layer for each word; start with 32 --> improve 64 --> improve 128)
        # input_length (the length of input sequence, same as above)                 
        Embedding(max_features, embedding_dims, input_length=maxlen), #embed each integer in a 128-dimensional vector
        # Performs the same function as Dropout, floating between 0 to 1 
        #(start with 0.1 --> improve 0.2 --> no further improve)
        SpatialDropout1D(0.2),
        LSTM(128, dropout=0.2, recurrent_dropout=0.0),
        # Sigmoid activation function to predict the probability for each class independently
        Dense(num_classes, activation='sigmoid'), #output layer will have 11 neurons (equal to the number of labels); 
                                                  #sigmoid activation function to predict the probability for each class independently
    ],
    name="LSTM_Model")
    return model

from keras.layers import Bidirectional

def getModel_bilstm(): # Using same parameters as stated above to build BiLSTM, which reverses the direction of information flow
    model = Sequential([
        Embedding(max_features, embedding_dims, input_length=maxlen),
        SpatialDropout1D(0.2),
        Bidirectional(LSTM(embedding_dims, dropout=0.2, recurrent_dropout=0.0)),
        Dense(num_classes, activation='sigmoid'), 
        ],
    name="BiLSTM_Model")
    return model   

**3.2 Run pipeline with different algorithms**

In [6]:
training_model = getModel_bilstm() # vs.getModel_lstm vs.getModel_cnn

# Since the corpus has been converted into a n-binary classification dataset, here we use binary-crosstropy loss function
training_model.compile(optimizer='adam', loss='binary_crossentropy', 
                       metrics= [tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.AUC(),
                                 tfa.metrics.F1Score(num_classes=num_classes, average='weighted')])
callbacks = [
    ReduceLROnPlateau(),
    ModelCheckpoint(filepath='model-conv1d.h5', save_best_only=True)
]

history = training_model.fit(
    X_train, y_train,
    class_weight=class_weight, # Using class weights as identifed above
    epochs=6, # A hyperparameter of gradient descent that controls the number of complete passes through the training dataset 
              #(start with 32 --> improve 16 --> improve 8 --> slight improve 6 --> no further improve)
    batch_size=32, # A hyperparameter of gradient descent that controls the number of training samples to work through 
                   #(start with 64 --> improve 32 --> no further improve)
    validation_split=0.3,
    callbacks=callbacks)

metrics = training_model.evaluate(X_test, y_test)
#print("{}: {}".format(training_model.metrics_names[1], metrics[1]))
#print("{}: {}".format(training_model.metrics_names[2], metrics[2]))
#print("{}: {}".format(training_model.metrics_names[3], metrics[3]))

# Save pre-trained model
# serialize model to JSON
ETC_bilstm_model_json = training_model.to_json()
with open("ETC_bilstm_model.json", "w") as json_file:
    json_file.write(ETC_bilstm_model_json)
# serialize weights to HDF5
training_model.save_weights("ETC_bilstm_model.h5")
print("Saved model to disk")

Epoch 1/6
601/601 [==============================] - 108s 168ms/step - loss: 2.0390e-04 - binary_accuracy: 0.8893 - auc: 0.7695 - f1_score: 0.2309 - val_loss: 0.2418 - val_binary_accuracy: 0.9112 - val_auc: 0.8654 - val_f1_score: 0.4116 - lr: 0.0010
Epoch 2/6
601/601 [==============================] - 87s 145ms/step - loss: 1.1959e-04 - binary_accuracy: 0.9373 - auc: 0.9267 - f1_score: 0.6497 - val_loss: 0.1435 - val_binary_accuracy: 0.9517 - val_auc: 0.9528 - val_f1_score: 0.7117 - lr: 0.0010
Epoch 3/6
601/601 [==============================] - 89s 149ms/step - loss: 8.8817e-05 - binary_accuracy: 0.9549 - auc: 0.9601 - f1_score: 0.7443 - val_loss: 0.1269 - val_binary_accuracy: 0.9557 - val_auc: 0.9657 - val_f1_score: 0.7339 - lr: 0.0010
Epoch 4/6
601/601 [==============================] - 122s 204ms/step - loss: 8.1579e-05 - binary_accuracy: 0.9576 - auc: 0.9683 - f1_score: 0.7698 - val_loss: 0.1224 - val_binary_accuracy: 0.9563 - val_auc: 0.9692 - val_f1_score: 0.7667 - lr: 0.0010
Ep

**4. Prediction** 

In [7]:
from keras.models import model_from_json
df_raw = pd.read_csv('haha_pred.csv') #QualificationPredictMe.SAV

df_raw['clean_text'] = [text_cleaner(t) for t in df_raw.stringEducation]
df_raw['clean_text_deep'] = df_raw.apply(lambda row: text_process(row.clean_text), axis=1)
df_raw['clean_text_deep_stem'] = df_raw['clean_text_deep'].apply(stem_sentences)

# load json and create model
json_file = open('ETC_bilstm_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("ETC_bilstm_model.h5")
print("Loaded model from disk")

tokenizer = Tokenizer(num_words=max_features, lower=True)
tokenizer.fit_on_texts(df_raw.clean_text_deep_stem)
sequences = tokenizer.texts_to_sequences(df_raw.clean_text_deep_stem)
x_new = pad_sequences(sequences, maxlen=maxlen)

df_new = pd.DataFrame(loaded_model.predict(x_new)) # .predict_classes/.predict_proba
df_new['annotation_main'] = df_new.idxmax(axis=1)

df_nn = df_raw.join(df_new)
df_nn = pd.DataFrame(df_nn)

df_nn = df_nn.rename(columns = {0: 'level1-4', 1: 'level5-6',
                                2: 'Level7BachelorDegree', 3: "PostgraduateBacheloHonours", 
                                4: "MastersDegree", 5: "DoctorateDegree", 
                                6: "OverseasSecondarySchoolQualification", 7:"NoQualification", 8:"invalidresponse"})

df_nn.drop(['clean_text', 'clean_text_deep', 'annotation_main'], axis=1, inplace=True)
df_nn = pd.DataFrame(df_nn)

# Set cut off scores (e.g., 50%)
#df_nn[df_nn.columns[3:12]] = df_nn[df_nn.columns[3:12]].mask((df_nn[df_nn.columns[3:12]] < .5), 0)

Loaded model from disk


In [12]:
pd.set_option('display.max_colwidth', None)

# Searching by likelihood value
# As the models produced a confidence score for every predicted QualType , we could set up a cut-off score, 
# e.g., asking models to return the predicted value only when the confidence score is greater than 0.5.
df_nn.loc[df_nn['PostgraduateBacheloHonours'] > .5].sort_values(by=['PostgraduateBacheloHonours'], ascending=False).head(5)

,stringEducation,clean_text_deep_stem,level1-4,level5-6,Level7BachelorDegree,PostgraduateBacheloHonours,MastersDegree,DoctorateDegree,OverseasSecondarySchoolQualification,NoQualification,invalidresponse
919,DIPLOMA PSYCHOPAEDIC NURSE AND TECHING DIPLOMA,diploma psychopaed nurs tech diploma,0.000513,0.000919,0.998227,0.000639,0.002014,0.001129,0.000001,0.000497,0.000779
2376,UNIVERSITY PAPERS AND A DIPLOMA,univers paper diploma,0.000922,0.000972,0.997097,0.000553,0.004954,0.002600,0.000005,0.001680,0.001725
904,DIPLOMA OF HORTICULTURE UNDERGRADUATE,diploma horticultur undergradu,0.001659,0.001538,0.996637,0.001534,0.002301,0.001764,0.000004,0.000566,0.001298
2358,UNIVERSITY DIPLOMA CERTIFICATE OF TEACHING,univers diploma certif teach,0.000448,0.002760,0.995994,0.000800,0.002243,0.000674,0.000004,0.000181,0.000570
1153,IB DIPLOMA CURRENTLY AT UNI,ib diploma current univers,0.000376,0.000625,0.995781,0.000157,0.013168,0.004818,0.000013,0.001855,0.002433


In [21]:
# Searching by key words e.g., bachelor/master/degree/honour
df_nn.loc[((df_nn['clean_text_deep_stem'].str.contains('bachelor')) & (df_nn['Level7BachelorDegree'] > .5))].sort_values(by=['Level7BachelorDegree'], ascending=False).head(5)

,stringEducation,clean_text_deep_stem,level1-4,level5-6,Level7BachelorDegree,PostgraduateBacheloHonours,MastersDegree,DoctorateDegree,OverseasSecondarySchoolQualification,NoQualification,invalidresponse
1091,HIGH SCHOOL DIPLOMA CURRENTLY DOING BACHELORS,high school diploma current bachelor,0.000706,0.007900,0.989372,0.003711,0.001782,0.001235,0.000018,0.000343,0.001022
2343,UNIVERSITY BACHELOR ARTS,univers bachelor art,0.000893,0.005934,0.986364,0.003781,0.001622,0.000654,0.000006,0.000099,0.000675
2159,TERTIARY UNIVERSITY BA,tertiari univers bachelor art,0.001345,0.005100,0.983459,0.005532,0.001003,0.000692,0.000011,0.000332,0.000946
272,BA DIPLOMA IN TEACHING,bachelor art diploma teach,0.000961,0.035877,0.977697,0.008837,0.000730,0.000166,0.000009,0.000037,0.000535
468,BACHELORS X 2,bachelor x 2,0.006297,0.019789,0.974795,0.004501,0.002323,0.000590,0.000025,0.000198,0.003199


In [22]:
df_nn.loc[df_nn['clean_text_deep_stem'].str.contains('lev')].sort_values(by=['level1-4'], ascending=False).head(5)

,stringEducation,clean_text_deep_stem,level1-4,level5-6,Level7BachelorDegree,PostgraduateBacheloHonours,MastersDegree,DoctorateDegree,OverseasSecondarySchoolQualification,NoQualification,invalidresponse
1603,NZ CERTIFICATE OF STUDY AND CAREER PREPARATION LEVEL 4,nz certif studi career prepar level4,0.983371,0.009203,0.004048,0.000866,0.000101,1.479990e-06,0.000286,0.003283,0.006600
648,CERTIFICATE IN PERSONAL TRAINING QND LEVEL 4 AGED CARE AND HEALTH,certif person train qnd level4 age care health,0.981800,0.008354,0.003692,0.000828,0.000086,1.075348e-06,0.000259,0.001983,0.004091
1600,NZ CERTIFICATE LEVEL 6,nz certif level6,0.981434,0.008411,0.003576,0.000996,0.000107,1.634062e-06,0.000260,0.003596,0.005528
1235,LEVEL 4 NZ CERTIFICATE IN ENGLISH LANGUAGE,level4 nz certif english languag,0.979000,0.007469,0.003647,0.000826,0.000138,2.452809e-06,0.000495,0.008318,0.009277
117,6TH FORM NCEA LV2,6th form ncea level2,0.977749,0.016722,0.002773,0.000955,0.000067,5.878023e-07,0.000251,0.001415,0.003097
